In [2]:
import pandas as pd
import numpy as np

In [3]:
cars = pd.read_csv('cars.csv',index_col=0)
cars.head()

,region,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,state,lat,long
13,denver,7995,2010.0,chevrolet,excellent,8 cylinders,gas,194050.0,clean,automatic,4wd,truck,white,co,39.8302,-105.0370
25,syracuse,4000,1995.0,dodge,excellent,8 cylinders,gas,133000.0,clean,automatic,4wd,truck,grey,ny,43.0734,-76.1568
28,greensboro,16000,2011.0,bmw,excellent,6 cylinders,gas,85000.0,salvage,automatic,fwd,sedan,grey,nc,35.5895,-82.5671
29,syracuse,10950,2011.0,buick,excellent,6 cylinders,gas,43418.0,clean,automatic,fwd,sedan,red,ny,43.1226,-76.1284
31,syracuse,9400,2011.0,bmw,good,6 cylinders,gas,145000.0,clean,automatic,4wd,SUV,blue,ny,43.1707,-76.0962


In [4]:
cars = cars[:5000]
cars = cars.reset_index()
cars = cars.drop(['index','region','lat','long','state'],axis=1)
cars

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color
0,7995,2010.0,chevrolet,excellent,8 cylinders,gas,194050.0,clean,automatic,4wd,truck,white
1,4000,1995.0,dodge,excellent,8 cylinders,gas,133000.0,clean,automatic,4wd,truck,grey
2,16000,2011.0,bmw,excellent,6 cylinders,gas,85000.0,salvage,automatic,fwd,sedan,grey
3,10950,2011.0,buick,excellent,6 cylinders,gas,43418.0,clean,automatic,fwd,sedan,red
4,9400,2011.0,bmw,good,6 cylinders,gas,145000.0,clean,automatic,4wd,SUV,blue
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,25900,2002.0,chevrolet,excellent,8 cylinders,gas,4422.0,clean,automatic,rwd,convertible,red
4996,5995,2013.0,ford,excellent,4 cylinders,gas,121000.0,clean,automatic,fwd,sedan,red
4997,20995,2013.0,ford,excellent,8 cylinders,gas,43635.0,clean,automatic,4wd,truck,grey
4998,15300,2014.0,gmc,excellent,4 cylinders,gas,54100.0,clean,automatic,fwd,SUV,red


In [5]:
cars['combination'] = cars.agg(lambda x: f"{x['price']} {x['condition']} {x['title_status']} {x['type']}", axis=1)

In [6]:
ordCond = {'excellent':4, 'good':3, 'like new':5, 'fair':2, 'new':6, 'salvage':1}
ordStatus = {'clean':6, 'salvage':1, 'rebuilt':4, 'lien':5, 'missing':3, 'parts only':2}

cars['condition_rank'] = cars['condition'].map(ordCond)
cars['status_rank'] = cars['title_status'].map(ordStatus)

In [7]:
cars.to_csv('cars1.csv')

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
cv = CountVectorizer()
cm = cv.fit_transform(cars['combination'])
simScore = cosine_similarity(cm.toarray())

In [10]:
price = 50000
typ = 'sedan'

In [12]:
carRecommend = cars.loc[(cars['price'] <= price) & (cars['type'] == typ)].sort_values(by=['price','condition_rank','status_rank'], ascending=False).index[0]
simCars = list(enumerate(simScore[carRecommend]))
sortCars = sorted(simCars, key=lambda x: x[1], reverse=True)

dfSim = []
for i in sortCars[:7]:
    dfSim.append(cars.iloc[i[0]])
dfSim = pd.DataFrame(dfSim)
dfSim[dfSim['price'] <= price].sort_values('price').sort_values(by='price')

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,type,paint_color,combination,condition_rank,status_rank
75,2500,2005.0,ford,good,6 cylinders,gas,140000.0,clean,automatic,fwd,sedan,blue,2500 good clean sedan,3,6
9,4999,2007.0,infiniti,good,6 cylinders,gas,113772.0,clean,automatic,rwd,sedan,custom,4999 good clean sedan,3,6
48,8500,2011.0,jaguar,good,8 cylinders,gas,95350.0,clean,automatic,rwd,sedan,silver,8500 good clean sedan,3,6
10,8599,2015.0,dodge,good,4 cylinders,gas,38484.0,clean,automatic,fwd,sedan,white,8599 good clean sedan,3,6
72,10499,2013.0,chrysler,good,6 cylinders,gas,88042.0,clean,automatic,rwd,sedan,grey,10499 good clean sedan,3,6
70,14999,2015.0,dodge,good,6 cylinders,gas,63037.0,clean,automatic,rwd,sedan,white,14999 good clean sedan,3,6
2572,33850,2014.0,mercedes-benz,good,8 cylinders,gas,75771.0,clean,automatic,rwd,sedan,black,33850 good clean sedan,3,6


In [1]:
# import joblib
# joblib.dump(simScore, 'score')